In [4]:
import pandas as pd 
import numpy as np

train = pd.read_csv("~/GitHub/LearningMachines/Prudential Challenge/train.csv")
test = pd.read_csv("~/GitHub/LearningMachines/Prudential Challenge/test.csv")
submission = pd.read_csv('/Users/davegelinas/Desktop/Kaggle/Prudential/13:12:2015/test.csv')

#We transform categorical values to dummies 0/1

categorical = ['Product_Info_1', 'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2', 'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Family_Hist_1', 'Medical_History_2', 'Medical_History_3', 'Medical_History_4', 'Medical_History_5', 'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 'Medical_History_9', 'Medical_History_10', 'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20', 'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26', 'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_30', 'Medical_History_31', 'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 'Medical_History_38', 'Medical_History_39', 'Medical_History_40', 'Medical_History_41','Medical_History_1', 'Medical_History_15', 'Medical_History_24', 'Medical_History_32']

from sklearn.feature_extraction import DictVectorizer
def one_hot_dataframe(data, cols, replace=False):
    """ Takes a dataframe and a list of columns that need to be encoded.
        Returns a 3-tuple comprising the data, the vectorized data,
        and the fitted vectorizor.
    """
    vec = DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)
    
train_ohd,_,_=one_hot_dataframe(train,categorical,replace=True)
test_ohd,_,_=one_hot_dataframe(test,categorical,replace=True)

features=train_ohd.columns.tolist()
features.remove("Id")
features.remove("Response")
train_features=train_ohd[features]
test_features=test_ohd[features]
train_features=train_features.fillna(-9999)
test_features=test_features.fillna(-9999)
print(train_features)

y_train = train["Response"].values

from sklearn import xgboost as xgb

#You can improve parameters if you want, this solution is not perfect at all

param = {'max_depth':6, 'eta':10**-1, 'silent':1, 'min_child_weight':3, 'subsample' : 0.7 ,"early_stopping_rounds":10,
          "objective"   : "count:poisson",'eval_metric': 'rmse','colsample_bytree':0.65}

num_round=700

dtrain=xgb.DMatrix(train_features,label=y_train)
dtest=xgb.DMatrix(test_features)

watchlist  = [(dtrain,'train')]

bst = xgb.train(param, dtrain, num_round, watchlist)

print("Training the model")
y_test_bst=bst.predict(dtest)

#we need integers to fit the model

def output_function(x):
    if x<1:
        return 1
    elif x>8:
        return 8
    elif int(round(x))==3:
        return 2
    else:
        return int(round(x))
        
y_test_bst_result=[output_function(y) for y in y_test_bst]

#write results
pred = clf.predict(test)
predClipped = np.clip(np.round(pred), 1, 8).astype(int) #Make the submissions within the accepted range

#submission = pd.read_csv('/Users/davegelinas/Desktop/Kaggle/Prudential/13:12:2015/test.csv')
submission["Response"] = predClipped
submission.to_csv('xgbPython.csv', index=False)

       Product_Info_4   Ins_Age        Ht        Wt       BMI  \
0            0.076923  0.641791  0.581818  0.148536  0.323008   
1            0.076923  0.059701  0.600000  0.131799  0.272288   
2            0.076923  0.029851  0.745455  0.288703  0.428780   
3            0.487179  0.164179  0.672727  0.205021  0.352438   
4            0.230769  0.417910  0.654545  0.234310  0.424046   
5            0.230769  0.507463  0.836364  0.299163  0.364887   
6            0.166194  0.373134  0.581818  0.173640  0.376587   
7            0.076923  0.611940  0.781818  0.403766  0.571612   
8            0.230769  0.522388  0.618182  0.184100  0.362643   
9            0.076923  0.552239  0.600000  0.284519  0.587796   
10           0.128205  0.537313  0.690909  0.309623  0.521668   
11           0.230769  0.298507  0.690909  0.271967  0.455050   
12           0.102564  0.567164  0.618182  0.163180  0.320784   
13           0.487179  0.223881  0.781818  0.361925  0.507515   
14           0.487179  0.

ImportError: cannot import name 'xgboost'